In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/stock_predict

/content/drive/MyDrive/stock_predict


In [3]:
import pandas as pd
df = pd.read_excel('SamsungResult_20220103-20230102.xlsx') #2022 1월~ 2023 1월까지의 뉴스

df = df[['일자', '제목', '키워드',
       '특성추출(가중치순 상위 50개)', '본문']]

samsung_stock_prices = pd.read_csv('samsung_2022-01-01~2023-02-02.csv') # 22년도 1월~ 23년도 2월까지의 삼성전자 주식가격

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
samsung_stock_prices

,날짜,종가,시가,고가,저가,거래량,변동 %
0,2023- 02- 02,"63,500","63,200","63,900","62,600",23.12M,2.75%
1,2023- 02- 01,"61,800","62,500","62,600","61,000",0.11K,1.31%
2,2023- 01- 31,"61,000","63,400","63,700","61,000",1.28K,-3.63%
3,2023- 01- 30,"63,300","64,900","64,900","63,100",19.15M,-2.01%
4,2023- 01- 29,"64,600","64,600","64,600","64,600",NaN,0.00%
...,...,...,...,...,...,...,...
320,2022- 01- 07,"78,300","78,100","78,400","77,400",15.10M,1.82%
321,2022- 01- 06,"76,900","76,700","77,600","76,700",8.25K,-0.65%
322,2022- 01- 05,"77,400","78,800","79,000","76,400",1.59K,-1.65%
323,2022- 01- 04,"78,700","78,800","79,200","78,300",3.28K,0.13%


In [4]:
samsung_stock_prices['날짜']= pd.to_datetime(samsung_stock_prices['날짜'])
samsung_stock_prices['날짜_하루뒤']= samsung_stock_prices['날짜']+ pd.DateOffset(days=1)
samsung_stock_prices['날짜_일주일뒤']=samsung_stock_prices['날짜']+pd.DateOffset(days=7) #일주일 뒤의 변동률을 계산할 수도 있을까봐 만들었습니다.

samsung_stock_prices

,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤
0,2023-02-02,"63,500","63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09
1,2023-02-01,"61,800","62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08
2,2023-01-31,"61,000","63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07
3,2023-01-30,"63,300","64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06
4,2023-01-29,"64,600","64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05
...,...,...,...,...,...,...,...,...,...
320,2022-01-07,"78,300","78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14
321,2022-01-06,"76,900","76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13
322,2022-01-05,"77,400","78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12
323,2022-01-04,"78,700","78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11


In [5]:
# '종가' 컬럼 데이터에서 쉼표(,) 제거하고 숫자로 변환
samsung_stock_prices['종가'] = samsung_stock_prices['종가'].str.replace(',', '').astype(float)

# '어제 종가' 컬럼 추가
#samsung_stock_prices['어제 종가'] = samsung_stock_prices['종가'].shift(-1)
samsung_stock_prices['내일 종가']= samsung_stock_prices['종가'].shift(1)
samsung_stock_prices

,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤,내일 종가
0,2023-02-02,63500.0,"63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09,NaN
1,2023-02-01,61800.0,"62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08,63500.0
2,2023-01-31,61000.0,"63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07,61800.0
3,2023-01-30,63300.0,"64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06,61000.0
4,2023-01-29,64600.0,"64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05,63300.0
...,...,...,...,...,...,...,...,...,...,...
320,2022-01-07,78300.0,"78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14,78300.0
321,2022-01-06,76900.0,"76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13,78300.0
322,2022-01-05,77400.0,"78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12,76900.0
323,2022-01-04,78700.0,"78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11,77400.0


In [6]:
# '하루변동률' 계산  '오늘'의 기사가 '내일'의 주가에 미친 영향을 보기 위해
samsung_stock_prices['하루변동률'] = (samsung_stock_prices['내일 종가'] - samsung_stock_prices['종가']) / samsung_stock_prices['종가'] *100
samsung_stock_prices


,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤,내일 종가,하루변동률
0,2023-02-02,63500.0,"63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09,NaN,NaN
1,2023-02-01,61800.0,"62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08,63500.0,2.750809
2,2023-01-31,61000.0,"63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07,61800.0,1.311475
3,2023-01-30,63300.0,"64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06,61000.0,-3.633491
4,2023-01-29,64600.0,"64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05,63300.0,-2.012384
...,...,...,...,...,...,...,...,...,...,...,...
320,2022-01-07,78300.0,"78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14,78300.0,0.000000
321,2022-01-06,76900.0,"76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13,78300.0,1.820546
322,2022-01-05,77400.0,"78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12,76900.0,-0.645995
323,2022-01-04,78700.0,"78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11,77400.0,-1.651842


In [7]:
# '일자' 열의 날짜 형식 변경 ('20230102' -> '2023-01-02')
df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
# '일자' 열의 날짜 형식 변경 ('2023-01-02' 형식으로)
df['일자'] = pd.to_datetime(df['일자']).dt.strftime('%Y-%m-%d')

# '날짜' 열의 날짜 형식 변경 ('2023-01-02' 형식으로)
samsung_stock_prices['날짜'] = pd.to_datetime(samsung_stock_prices['날짜']).dt.strftime('%Y-%m-%d')

In [ ]:
samsung_stock_prices

,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤,내일 종가,하루변동률
0,2023-02-02,63500.0,"63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09,NaN,NaN
1,2023-02-01,61800.0,"62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08,63500.0,2.750809
2,2023-01-31,61000.0,"63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07,61800.0,1.311475
3,2023-01-30,63300.0,"64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06,61000.0,-3.633491
4,2023-01-29,64600.0,"64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05,63300.0,-2.012384
...,...,...,...,...,...,...,...,...,...,...,...
320,2022-01-07,78300.0,"78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14,78300.0,0.000000
321,2022-01-06,76900.0,"76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13,78300.0,1.820546
322,2022-01-05,77400.0,"78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12,76900.0,-0.645995
323,2022-01-04,78700.0,"78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11,77400.0,-1.651842


In [ ]:
df.head()

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 '반도체 초강대국' 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ..."
1,2023-01-02,“삼성전자 SK하이닉스 주가 올해는 괜찮다” 믿어도 될까 [이종화의 세돌아이],"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아..."
2,2023-01-02,삼성전자 시무식 개최...한종희 “다시 한번 한계의 벽을 넘자”,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디..."
3,2023-01-02,"한종희 삼성전자 부회장 ""다시 한번 한계의 벽 넘자""","부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...
4,2023-01-02,"삼성전자 한종희 경계현 대표 ""과감한 도전과 변신으로 도약 전환점 만들자""","삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...


In [8]:
merged_df = pd.merge(df,samsung_stock_prices,how='left',left_on='일자',right_on='날짜') #날짜를 기준으로 기사 파일과  주가 파일을 병합
df['변동률'] = merged_df['하루변동률'] #병합 후 '하루 변동률' 컬럼만 붙이기
df

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문,변동률
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 '반도체 초강대국' 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ...",-0.180180
1,2023-01-02,“삼성전자 SK하이닉스 주가 올해는 괜찮다” 믿어도 될까 [이종화의 세돌아이],"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아...",-0.180180
2,2023-01-02,삼성전자 시무식 개최...한종희 “다시 한번 한계의 벽을 넘자”,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디...",-0.180180
3,2023-01-02,"한종희 삼성전자 부회장 ""다시 한번 한계의 벽 넘자""","부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...,-0.180180
4,2023-01-02,"삼성전자 한종희 경계현 대표 ""과감한 도전과 변신으로 도약 전환점 만들자""","삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...,-0.180180
...,...,...,...,...,...,...
5429,2022-01-03,"'꼰대 문화' 버려라, 실패해도 좋아 삼성전자 파격 신년사","꼰대,문화,실패,신년사,삼성전자,파격,과거,비즈니스,모델,전략,경직,프로세스,시대,...","임직원,삼성전자,부회장,사업부장,신년사,시무식,한종희,경계현,노태문,이재승,이정배,...","[머니투데이 심재현 기자] \n""과거의 비즈니스 모델과 전략, 경직된 프로세스와 시...",0.127226
5430,2022-01-03,한종희 경계현 삼성전자 대표 “가치있는 고객경험으로 사업 품격 높이자”,"한종희,경계현,가치,삼성전자,대표,고객경험,사업,품격,기업,신년사,삼성전자,최고,경...","삼성전자,임직원,사업부장,부회장,한종희,경계현,시무식,신년사,노태문,이재승,최시영,...",삼성전자 최고경영자(CEO)들이 가치 있는 고객경험을 통해 사업의 품격을 높여나가자...,0.127226
5431,2022-01-03,"1Q 반도체 가격, 삼성전자 시안 공장에 달렸다","가격,반도체,공장,삼성전자,시안,삼성전자,공장,중국,시안,록다운,환경,반도체,가격,...","반도체,삼성전자,연구원,낸드플래시,삼성증권,자릿수,공급사,실리콘웨이퍼,생산능력,생산...",[머니투데이 정인지 기자] 삼성전자의 중국 시안 공장이 록다운되면서 반도체 가격 환...,0.127226
5432,2022-01-03,"삼성전자, 메모리 초격차 스마트폰 가전 시너지로 매출 300조시대 연다","삼성전자,메모리,초격차,스마트폰,가전,시너지,매출,시대,연다,재계,삼성전자,가격,반...","반도체,삼성전자,다운사이클,차세대,경쟁력,산업은행,스마트폰",◆ 2022 재계가 뛴다 ◆\n 삼성전자는 올해 가격 반등이 전망되는 메모리 반도체...,0.127226


In [9]:
# '변동률' 열의 NaN 값을 0으로 대체.  자료가 짤려있어서 Nan 값이 있음...
df['변동률'].fillna(0, inplace=True)
df

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문,변동률
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 '반도체 초강대국' 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ...",-0.180180
1,2023-01-02,“삼성전자 SK하이닉스 주가 올해는 괜찮다” 믿어도 될까 [이종화의 세돌아이],"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아...",-0.180180
2,2023-01-02,삼성전자 시무식 개최...한종희 “다시 한번 한계의 벽을 넘자”,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디...",-0.180180
3,2023-01-02,"한종희 삼성전자 부회장 ""다시 한번 한계의 벽 넘자""","부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...,-0.180180
4,2023-01-02,"삼성전자 한종희 경계현 대표 ""과감한 도전과 변신으로 도약 전환점 만들자""","삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...,-0.180180
...,...,...,...,...,...,...
5429,2022-01-03,"'꼰대 문화' 버려라, 실패해도 좋아 삼성전자 파격 신년사","꼰대,문화,실패,신년사,삼성전자,파격,과거,비즈니스,모델,전략,경직,프로세스,시대,...","임직원,삼성전자,부회장,사업부장,신년사,시무식,한종희,경계현,노태문,이재승,이정배,...","[머니투데이 심재현 기자] \n""과거의 비즈니스 모델과 전략, 경직된 프로세스와 시...",0.127226
5430,2022-01-03,한종희 경계현 삼성전자 대표 “가치있는 고객경험으로 사업 품격 높이자”,"한종희,경계현,가치,삼성전자,대표,고객경험,사업,품격,기업,신년사,삼성전자,최고,경...","삼성전자,임직원,사업부장,부회장,한종희,경계현,시무식,신년사,노태문,이재승,최시영,...",삼성전자 최고경영자(CEO)들이 가치 있는 고객경험을 통해 사업의 품격을 높여나가자...,0.127226
5431,2022-01-03,"1Q 반도체 가격, 삼성전자 시안 공장에 달렸다","가격,반도체,공장,삼성전자,시안,삼성전자,공장,중국,시안,록다운,환경,반도체,가격,...","반도체,삼성전자,연구원,낸드플래시,삼성증권,자릿수,공급사,실리콘웨이퍼,생산능력,생산...",[머니투데이 정인지 기자] 삼성전자의 중국 시안 공장이 록다운되면서 반도체 가격 환...,0.127226
5432,2022-01-03,"삼성전자, 메모리 초격차 스마트폰 가전 시너지로 매출 300조시대 연다","삼성전자,메모리,초격차,스마트폰,가전,시너지,매출,시대,연다,재계,삼성전자,가격,반...","반도체,삼성전자,다운사이클,차세대,경쟁력,산업은행,스마트폰",◆ 2022 재계가 뛴다 ◆\n 삼성전자는 올해 가격 반등이 전망되는 메모리 반도체...,0.127226


In [10]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.2 MB/s eta 0:00:00


뉴스 '제목'을 활용한 감성 분석.


In [11]:
from konlpy.tag import Okt
import re
import numpy as np
okt = Okt()
n_ = []
title_rename = []
for i in range(len(df)):
  if(i % 10000 ==0):
    print(i,"단계 완료")
  title_rename.append(re.sub("[\(\[].*?[\)\]]", "",df.iloc[i]['제목']))  #본문 내용은 양이 많아 실험적으로 하기위해 제목으로만 진행.
  n_.append(' '.join(okt.nouns(df.iloc[i]['제목'])))
df['nouns'] = n_
df['제목']=title_rename
df = df[df['nouns']!='']

#2차 불용어 제거
df['제목'] = df['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['제목'].replace('', np.nan, inplace=True)
df = df.dropna(how='any')



0 단계 완료


<ipython-input-11-63462de7e6fd>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['제목'] = df['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [12]:
from collections import Counter

# 'nouns' 열에서 모든 단어 리스트 추출
all_words = [word for sublist in df['nouns'].str.split(' ').dropna() for word in sublist]

# 각 단어의 빈도수를 세기
word_counts = dict(Counter(all_words))


word_counts



{'삼성': 5267,
 '전자': 5519,
 '선제': 4,
 '투자': 182,
 '기술': 172,
 '차별': 6,
 '화로': 2,
 '반도체': 627,
 '초강대국': 2,
 '하이닉스': 225,
 '주가': 226,
 '올해': 99,
 '이종화': 5,
 '돌아이': 5,
 '시무': 2,
 '개최': 79,
 '한종': 115,
 '다시': 65,
 '한번': 11,
 '한계': 6,
 '벽': 8,
 '부회장': 133,
 '경계': 41,
 '현': 44,
 '대표': 32,
 '과감': 1,
 '도전': 14,
 '변신': 6,
 '도약': 5,
 '전환': 15,
 '점': 31,
 '환경': 29,
 '경영': 43,
 '전략': 57,
 '본격': 35,
 '원년': 6,
 '신년사': 4,
 '주도': 10,
 '홀딩스': 2,
 '한화': 10,
 '솔루션': 49,
 '셀트리온': 3,
 '두': 12,
 '번': 6,
 '실망': 2,
 '데이터': 14,
 '증시': 51,
 '기관': 51,
 '외국인': 94,
 '코스피': 168,
 '주간': 3,
 '매수': 110,
 '위': 280,
 '해킹': 14,
 '급': 4,
 '상장': 12,
 '철회': 2,
 '탈': 4,
 '임인년': 2,
 '빅딜': 4,
 '무소식': 2,
 '인': 24,
 '까닭': 12,
 '권오현': 2,
 '고문': 14,
 '유명': 18,
 '통상': 8,
 '본부장': 6,
 '서울대': 9,
 '이사': 25,
 '선출': 4,
 '애플': 43,
 '울': 5,
 '마오타이': 1,
 '선방': 40,
 '이재용': 158,
 '회장': 89,
 '입사': 11,
 '뉴': 15,
 '지휘': 1,
 '벤츠': 1,
 '디자이너': 2,
 '출신': 24,
 '이일환': 1,
 '부사': 52,
 '입': 40,
 '박': 2,
 '동남아': 4,
 '출장': 11,
 '귀국': 5,
 '호': 2,
 '스핀오프': 2

In [13]:
vocab= {}
cnt=0
for i in df['nouns']:
  i= i.split(' ')
  for j in range(len(i)):
    if i[j] in vocab or len(i[j])<=1:
      cnt=cnt+1
      pass
    else:
      vocab[i[j]]=0
print(len(vocab),vocab)


4239 {'삼성': 0, '전자': 0, '선제': 0, '투자': 0, '기술': 0, '차별': 0, '화로': 0, '반도체': 0, '초강대국': 0, '하이닉스': 0, '주가': 0, '올해': 0, '이종화': 0, '돌아이': 0, '시무': 0, '개최': 0, '한종': 0, '다시': 0, '한번': 0, '한계': 0, '부회장': 0, '경계': 0, '대표': 0, '과감': 0, '도전': 0, '변신': 0, '도약': 0, '전환': 0, '환경': 0, '경영': 0, '전략': 0, '본격': 0, '원년': 0, '신년사': 0, '주도': 0, '홀딩스': 0, '한화': 0, '솔루션': 0, '셀트리온': 0, '실망': 0, '데이터': 0, '증시': 0, '기관': 0, '외국인': 0, '코스피': 0, '주간': 0, '매수': 0, '해킹': 0, '상장': 0, '철회': 0, '임인년': 0, '빅딜': 0, '무소식': 0, '까닭': 0, '권오현': 0, '고문': 0, '유명': 0, '통상': 0, '본부장': 0, '서울대': 0, '이사': 0, '선출': 0, '애플': 0, '마오타이': 0, '선방': 0, '이재용': 0, '회장': 0, '입사': 0, '지휘': 0, '벤츠': 0, '디자이너': 0, '출신': 0, '이일환': 0, '부사': 0, '동남아': 0, '출장': 0, '귀국': 0, '스핀오프': 0, '솔티드': 0, '브릿지': 0, '유치': 0, '해령': 0, '하이엔드': 0, '테크': 0, '나노': 0, '가지': 0, '이야기': 0, '물폭탄': 0, '생명': 0, '부꾸미': 0, '손실': 0, '지금': 0, '아야': 0, '세계': 0, '최고': 0, '치명': 0, '악몽': 0, '카오': 0, '초토화': 0, '출고': 0, '사업': 0, '부진': 0, '고민': 0, '해설': 0, '최악': 0, '기회': 0, '스

In [14]:
# 'updown' 열 추가
df['updown'] = df['변동률'].apply(lambda x: 1 if x > 0 else 0) #변동률을 직접 집어넣고 싶었으나... 변동률이 0보다크면(다음날 증가했다면) updown을 1로 설정

In [15]:
df

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문,변동률,nouns,updown
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 반도체 초강대국 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ...",-0.180180,삼성 전자 선제 투자 기술 차별 화로 반도체 초강대국,0
1,2023-01-02,삼성전자 하이닉스 주가 올해는 괜찮다 믿어도 될까,"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아...",-0.180180,삼성 전자 하이닉스 주가 올해 이종화 돌아이,0
2,2023-01-02,삼성전자 시무식 개최한종희 다시 한번 한계의 벽을 넘자,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디...",-0.180180,삼성 전자 시무 개최 한종 다시 한번 한계 벽,0
3,2023-01-02,한종희 삼성전자 부회장 다시 한번 한계의 벽 넘자,"부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...,-0.180180,한종 삼성 전자 부회장 다시 한번 한계 벽,0
4,2023-01-02,삼성전자 한종희 경계현 대표 과감한 도전과 변신으로 도약 전환점 만들자,"삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...,-0.180180,삼성 전자 한종 경계 현 대표 과감 도전 변신 도약 전환 점,0
...,...,...,...,...,...,...,...,...
5429,2022-01-03,꼰대 문화 버려라 실패해도 좋아 삼성전자 파격 신년사,"꼰대,문화,실패,신년사,삼성전자,파격,과거,비즈니스,모델,전략,경직,프로세스,시대,...","임직원,삼성전자,부회장,사업부장,신년사,시무식,한종희,경계현,노태문,이재승,이정배,...","[머니투데이 심재현 기자] \n""과거의 비즈니스 모델과 전략, 경직된 프로세스와 시...",0.127226,꼰대 문화 삼성 전자 파격 신년사,1
5430,2022-01-03,한종희 경계현 삼성전자 대표 가치있는 고객경험으로 사업 품격 높이자,"한종희,경계현,가치,삼성전자,대표,고객경험,사업,품격,기업,신년사,삼성전자,최고,경...","삼성전자,임직원,사업부장,부회장,한종희,경계현,시무식,신년사,노태문,이재승,최시영,...",삼성전자 최고경영자(CEO)들이 가치 있는 고객경험을 통해 사업의 품격을 높여나가자...,0.127226,한종 경계 현 삼성 전자 대표 가치 고객 경험 사업 품격 높이,1
5431,2022-01-03,반도체 가격 삼성전자 시안 공장에 달렸다,"가격,반도체,공장,삼성전자,시안,삼성전자,공장,중국,시안,록다운,환경,반도체,가격,...","반도체,삼성전자,연구원,낸드플래시,삼성증권,자릿수,공급사,실리콘웨이퍼,생산능력,생산...",[머니투데이 정인지 기자] 삼성전자의 중국 시안 공장이 록다운되면서 반도체 가격 환...,0.127226,반도체 가격 삼성 전자 시안 공장,1
5432,2022-01-03,삼성전자 메모리 초격차 스마트폰 가전 시너지로 매출 조시대 연다,"삼성전자,메모리,초격차,스마트폰,가전,시너지,매출,시대,연다,재계,삼성전자,가격,반...","반도체,삼성전자,다운사이클,차세대,경쟁력,산업은행,스마트폰",◆ 2022 재계가 뛴다 ◆\n 삼성전자는 올해 가격 반등이 전망되는 메모리 반도체...,0.127226,삼성 전자 메모리 초 격차 스마트폰 가전 시너지 매출 연,1


In [16]:
# 'updown' 열에서 값이 0인 개수를 세기
count_zeros = df[df['updown'] == 0].shape[0]
print("updown 열에서 0의 개수:", count_zeros)
print('나머지 1의 개수 ', df.shape[0]-count_zeros)  #0이 더 많은건 Nan을 0으로 처리해서 그런듯

updown 열에서 0의 개수: 3538
나머지 1의 개수  1896


In [17]:
up = df.shape[0]-count_zeros
down = count_zeros
up_ratio = up/(up+down)
down_ratio = down/(up+down)

import collections
for i,w in enumerate(df['nouns']):

    w = w.split(' ')
    if (df.iloc[i]['updown']==1):
        for j in range(len(w)):
            noun = w[j]

            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]

            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

vocab

{'삼성': 5.268678689738198,
 '전자': 4.342289289666055,
 '선제': -0.39565697460434307,
 '투자': -0.5023923444976031,
 '기술': 3.986750092013258,
 '차별': -0.09348546190651469,
 '화로': -0.6978284873021715,
 '반도체': -17.769230769230827,
 '초강대국': -0.6978284873021715,
 '하이닉스': 4.494295178505716,
 '주가': -19.854619065145457,
 '올해': 2.457489878542513,
 '이종화': 0.2554287817445711,
 '돌아이': 0.2554287817445711,
 '시무': -0.6978284873021715,
 '개최': 8.435774751564228,
 '한종': 8.874861980125141,
 '다시': -1.679425837320573,
 '한번': -0.8380566801619435,
 '한계': -2.0934854619065146,
 '부회장': 0.5944055944055986,
 '경계': -1.3054839896945145,
 '대표': 0.8347442031652569,
 '과감': -0.34891424365108575,
 '도전': 1.1152005888847998,
 '변신': -0.09348546190651463,
 '도약': 0.2554287817445711,
 '전환': -0.23371365476628653,
 '환경': 9.881486934118513,
 '경영': 9.996687523003313,
 '전략': 4.111888111888113,
 '본격': -5.211998527788,
 '원년': -2.0934854619065146,
 '신년사': 1.6043430253956568,
 '주도': -2.4891424365108574,
 '홀딩스': 0.30217151269782844,
 '한화': 1.

In [26]:
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=False)
sorted_vocab

[('주가', -19.854619065145457),
 ('반도체', -17.769230769230827),
 ('사장', -15.009937430990064),
 ('목표', -12.567537725432457),
 ('하락', -11.218623481781373),
 ('에너지', -11.003312476996687),
 ('갤럭시', -9.91976444608023),
 ('이재용', -9.128450496871544),
 ('최고', -8.844681634155314),
 ('노조', -8.819654030180343),
 ('선정', -8.701140964298858),
 ('시스템', -8.211998527788001),
 ('인사', -8.211998527788001),
 ('관리', -8.07177033492823),
 ('직원', -7.5391976444608),
 ('세탁기', -7.373941847626058),
 ('회복', -7.143540669856457),
 ('협력', -6.826278984173717),
 ('추진', -6.7695988222304),
 ('싱스', -6.377254324622744),
 ('이익', -6.121825542878172),
 ('신저', -6.081707765918288),
 ('작년', -6.075082811924915),
 ('여성', -5.978284873021715),
 ('유출', -5.978284873021715),
 ('업데이트', -5.931542142068458),
 ('특징', -5.801251380198743),
 ('출시', -5.732793522267199),
 ('임원', -5.6543982333456),
 ('화학물질', -5.582627898417372),
 ('초반', -5.560912771439086),
 ('핵심', -5.467427309532572),
 ('영업', -5.430622009569415),
 ('스마트', -5.358851674641143),
 ('재고

In [18]:
vocab['삼성']=0
vocab['전자']=0

In [19]:
total = []
for i,w  in enumerate(df['nouns']):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab):
          continue
        else:
          sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
df['sent_score'] = total

In [20]:
df['sent_score'].mean()

-0.13264989408938238

In [21]:
df

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문,변동률,nouns,updown,sent_score
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 반도체 초강대국 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ...",-0.180180,삼성 전자 선제 투자 기술 차별 화로 반도체 초강대국,0,-1.796630
1,2023-01-02,삼성전자 하이닉스 주가 올해는 괜찮다 믿어도 될까,"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아...",-0.180180,삼성 전자 하이닉스 주가 올해 이종화 돌아이,0,-1.770282
2,2023-01-02,삼성전자 시무식 개최한종희 다시 한번 한계의 벽을 넘자,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디...",-0.180180,삼성 전자 시무 개최 한종 다시 한번 한계 벽,0,1.333538
3,2023-01-02,한종희 삼성전자 부회장 다시 한번 한계의 벽 넘자,"부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...,-0.180180,한종 삼성 전자 부회장 다시 한번 한계 벽,0,0.607287
4,2023-01-02,삼성전자 한종희 경계현 대표 과감한 도전과 변신으로 도약 전환점 만들자,"삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...,-0.180180,삼성 전자 한종 경계 현 대표 과감 도전 변신 도약 전환 점,0,0.758220
...,...,...,...,...,...,...,...,...,...
5429,2022-01-03,꼰대 문화 버려라 실패해도 좋아 삼성전자 파격 신년사,"꼰대,문화,실패,신년사,삼성전자,파격,과거,비즈니스,모델,전략,경직,프로세스,시대,...","임직원,삼성전자,부회장,사업부장,신년사,시무식,한종희,경계현,노태문,이재승,이정배,...","[머니투데이 심재현 기자] \n""과거의 비즈니스 모델과 전략, 경직된 프로세스와 시...",0.127226,꼰대 문화 삼성 전자 파격 신년사,1,0.929886
5430,2022-01-03,한종희 경계현 삼성전자 대표 가치있는 고객경험으로 사업 품격 높이자,"한종희,경계현,가치,삼성전자,대표,고객경험,사업,품격,기업,신년사,삼성전자,최고,경...","삼성전자,임직원,사업부장,부회장,한종희,경계현,시무식,신년사,노태문,이재승,최시영,...",삼성전자 최고경영자(CEO)들이 가치 있는 고객경험을 통해 사업의 품격을 높여나가자...,0.127226,한종 경계 현 삼성 전자 대표 가치 고객 경험 사업 품격 높이,1,2.257116
5431,2022-01-03,반도체 가격 삼성전자 시안 공장에 달렸다,"가격,반도체,공장,삼성전자,시안,삼성전자,공장,중국,시안,록다운,환경,반도체,가격,...","반도체,삼성전자,연구원,낸드플래시,삼성증권,자릿수,공급사,실리콘웨이퍼,생산능력,생산...",[머니투데이 정인지 기자] 삼성전자의 중국 시안 공장이 록다운되면서 반도체 가격 환...,0.127226,반도체 가격 삼성 전자 시안 공장,1,-2.591461
5432,2022-01-03,삼성전자 메모리 초격차 스마트폰 가전 시너지로 매출 조시대 연다,"삼성전자,메모리,초격차,스마트폰,가전,시너지,매출,시대,연다,재계,삼성전자,가격,반...","반도체,삼성전자,다운사이클,차세대,경쟁력,산업은행,스마트폰",◆ 2022 재계가 뛴다 ◆\n 삼성전자는 올해 가격 반등이 전망되는 메모리 반도체...,0.127226,삼성 전자 메모리 초 격차 스마트폰 가전 시너지 매출 연,1,1.881671


In [24]:
for index,i in enumerate(df['sent_score']):
  if i >5 : #감성지수가 0보다 큰 (상대적으로 긍정적인) 기사 제목
    print(df.iloc[index]['제목'],i)

삼성전자 나노급 램 개발 마쳐 내년께 양산 5.710973237289012
한종희 삼성전자 부회장 베트남 주석 만나 부산엑스포 지지 요청 5.72307692307692
삼성전자 경영진 베트남 국가주석 접견   부산엑스포 유치 지원 5.837762237762235
삼성전자 베트남 국가주석 접견  부산엑스포 유치 지원 5.8844722528933024
  총회 열리는 파리서 삼성전자 부산엑스포 홍보 5.133373205741623
삼성전자  총회 열리는 파리에서 부산엑스포 홍보전 5.023187338976808
삼성전자  총회 열리는 파리서 부산엑스포 유치 홍보 5.461216415163779
한종희 삼성전자 부회장 오스트리아에 부산엑스포 지지 요청 6.2298695456590165
삼성전자 한종희 부회장 오스트리아 외교부에 부산엑스포 지지 요청 5.997534044902463
삼성전자 오스트리아 외교부 접견  부산엑스포 지지 요청 6.603811393285073
삼성전자 오스트리아 외교부 접견  부산엑스포 지지 요청 6.603811393285073
삼성전자 시간 벌었다 나노 양산 연기 5.653048705680266
삼성전자 남아프리카공화국서 부산엑스포 유치 지원 6.001472211998524
삼성전자 내년 세대 나노급 램 양산 세계 최초 5.202960781908139
 삼성전자 년 나노 양산개시 7.844092749355886
삼성전자 엘살바도르에 부산엑스포 지지 요청 6.999316473000678
삼성전자 엘살바도르 외교장관 접견  부산엑스포 지지 요청 6.326904674273092
삼성전자 엘살바도르 외교부 장관 만나  부산엑스포 지지 요청 6.19294156136261
삼성전자 엘살바도르 외교부장관 접견 부산엑스포 유치 지원 6.028413691571584
삼성전자 년 나노 파운드리 양산 8.959293338240686
삼성전자 나노 년 양산 선언  6.186439291702438
파운드리도 초격차 삼성전자 년 나노 시대 선언 5.020472948

In [23]:
for index,i in enumerate(df['sent_score']):
  if i <-10 : #감성지수가 20보다 작은(부정적인 기사)
    print(df.iloc[index]['제목'],i)